# Unitary propagator Error estimation for first-order Trotter formulas

Hamiltonian unitary evolution is part of the Quantum Phase Estimation (QPE) algorithm for calculation of energy eigenvalues of a target Hamiltonian $H$, in a digital quantum computer. In practice, given the classically intractable diagonalization of $H$, approximations are need for the implementation of the unitary time evolution operator. One of these common approximations is the (so-called first order) Trotter approximation:
$$
U(t)=e^{-iHt}=\prod_{n=1}^{N}e^{-iH_{n}t}+\mathcal{A}=U_{T}(t)+\mathcal{A}
$$
where $H=\sum_{n}H_{n}$ and $H_{n}$ can be classically diagonalized for all $n$. The deviation of the Trotterized evolution operator from the exact one $U_{T}(t)-U(t)=\mathcal{A}$ can be quantified in terms of its spectral norm:
$$
|| \mathcal{A}||=\alpha t^{2}+\mathcal{O}(t^{3})
$$
$$
\alpha=\sum_{i,j}|| [H_{i},H_{j}]||
$$
It has been shown in the literature (see for instance [PNAS **29**, 7555 (2017)](https://www.pnas.org/doi/abs/10.1073/pnas.1619152114), [Quantum **4**, 296 (2020)](https://quantum-journal.org/papers/q-2020-07-16-296/)) that the Trotter error $\alpha$ is an important figure of merit to quantify an upper-bound estimation of the error incurred in estimation of energy eigenvalues within a QPE framework, which ultimately determines the resources needed to build a quantum computer able to compute energy eigenvalues within a target error.

# Trotter error for grouping techniques of molecular electronic Hamiltonians

For potential applications of quantum computers in quantum chemistry, we have the calculation of energy eigenvalues of molecular electronic Hamiltonians, which in second-quantized form are given by
$$
H_{el}=\sum_{pq}h_{pq}a^{\dagger}_{p}a_{q}+\sum_{pqrs}g_{pqrs}a^{\dagger}_{p}a_{q}a^{\dagger}_{r}a_{s}
$$
$a_{p}$ ($a^{\dagger}_{p}$) being the annhilitation (creation) fermionic operator that acts on the $p$th fermionic mode (molecular orbital). Considering an estimation of energy eigenvalues with QPE and the Trotter approximation, our first task is to decompose the electronic Hamiltonian into classically diagonalizable (or fast-forwardable) Hamiltonian fragments. 


Once these Hamiltonian fragments are found, we can use Tequila to compute $\alpha$ and compare the performance of different Hamiltonian decomposition techniques for test systems. Smaller $\alpha$'s imply smaller (worst-case scenario) scaling of error in phase estimation with circuit depth. Thus, it is a signature of calculation speedup in a quantum computer and a more resource-efficient quantum circuit.

Below we show how to perform a partition of the Hamiltonian using three techniques, namely Fully-Commuting grouping ([JCTC **16**, 2400 (2020)](https://pubs.acs.org/doi/10.1021/acs.jctc.0c00008)) using the so-called Largest-First (FC-LF) and Sorted-Insertion (FC-SI) heuristics; and Low-Rank (LR) factorization technique ([npj Quantum Information **7**, 83 (2021)](https://www.nature.com/articles/s41534-021-00416-z)).

In [3]:
from tequila.trotter_err.main_trot import EstTrotErr
import numpy as np
import tequila as tq
from tequila.hamiltonian import QubitHamiltonian, PauliString
from tequila.grouping.binary_rep import BinaryPauliString, BinaryHamiltonian
from tequila.grouping.fermionic_methods import do_svd
from tequila.grouping.fermionic_functions import obt_to_ferm, convert_tbts_to_frags
from tequila.grouping.fermionic_functions import n_elec
import openfermion

In [18]:
def build_toymol():
    '''
    Build the qubit Hamiltonian with Jordan Wigner encoding for H2 at a particular geometry.
    '''

    trafo = "JordanWigner"
    mol = tq.chemistry.Molecule(
                            geometry="H 0.0 0.0 0.0 \n H 0.0 0.0 1.0",
                            basis_set="sto3g",
                            transformation=trafo,
                            backend='pyscf'
                                 )
    H = mol.make_hamiltonian()
    
    Hferm = mol.make_molecular_hamiltonian()
    
    return H,openfermion.get_fermion_operator(Hferm)

def get_fclf(H):
    '''
    Get the Hamiltonian fragments under the qubit FC-LF partition method, as explained in JCTC 16, 2400 (2020))
    '''
    
    Hbin = BinaryHamiltonian.init_from_qubit_hamiltonian(H)
    options={"method":"lf", "condition": "fc"}
    
    commuting_parts, _ = Hbin.commuting_groups(options=options)
    print("Number of FC-LF groups: {}".format(len(commuting_parts)))
    
    ListFrags=[]

    for i in range(len(commuting_parts)):
        tqqubFrag=commuting_parts[i].to_qubit_hamiltonian()
        ListFrags.append(tqqubFrag.to_openfermion())
        
    return ListFrags

def get_fcsi(H):
    '''
    Get the Hamiltonian fragments under the qubit FC-SI partition method, as explained in JCTC 16, 2400 (2020))
    '''
    
    Hbin = BinaryHamiltonian.init_from_qubit_hamiltonian(H)
    options={"method":"si", "condition": "fc"}
    
    commuting_parts, _ = Hbin.commuting_groups(options=options)
    print("Number of FC-SI groups: {}".format(len(commuting_parts)))
    
    ListFrags=[]

    for i in range(len(commuting_parts)):
        tqqubFrag=commuting_parts[i].to_qubit_hamiltonian()
        ListFrags.append(tqqubFrag.to_openfermion())
        
    return ListFrags

#set opf routines to get LR fragments...
def cartan_to_fermionic_operator(cobt, ctbts, orb_rot):
    '''
    Turn Hamiltonian fragments in their diagonal form into OpenFermion's FermionOperator.
    '''
    obt = np.einsum("pa, qb, ab", orb_rot[0], orb_rot[0], cobt)
    tbts = np.einsum("ipa, iqb, irc, isd, iabcd -> ipqrs", 
                     orb_rot[1:], orb_rot[1:], orb_rot[1:], orb_rot[1:], ctbts)
    ferm_ops = [obt_to_ferm(obt,True)] + convert_tbts_to_frags(tbts, True)
    return ferm_ops

def get_LR(Hferm,name="h2"):
    '''
    Get Low-Rank Hamiltonian fragments, according to the technique outlined in npj Quantum Information 7, 83 (2021)
    '''
    
    orb_rots, cartan_obt, cartan_tbts, meas_alloc = do_svd(Hferm, n_elec(name))
    
    ferm_ops = cartan_to_fermionic_operator(cartan_obt, cartan_tbts, orb_rots)
    
    return ferm_ops


In [19]:
nqubs=openfermion.count_qubits(Hferm)

In [20]:
#Build the toy molecule and compute FC-SI, FC-LF and LR Hamiltonian fragments.
Hq,Hferm=build_toymol()

FCLFFrags=get_fclf(Hq)

FCSIFrags=get_fcsi(Hq)

LRFrags=get_LR(Hferm)

converged SCF energy = -1.06610864931794
Number of FC-LF groups: 2
Number of FC-SI groups: 2
Starting SVD routine
Diagonalizing two-body tensor
Truncating SVD for coefficients with magnitude smaller or equal to 2.7755575615628914e-17, using 3 fragments
Finished SVD routine
Allocating measurements


In [25]:
#Calculation of Trotter error for FC-LF fragments:
EstTrotErr(FCLFFrags,nqubs)

0.4211769529694661

In [26]:
#Calculation of Trotter error for FC-SI fragments:
EstTrotErr(FCSIFrags,nqubs)

0.42117695296946633

In [24]:
#Calculation of Trotter error for LR fragments
EstTrotErr(LRFrags,nqubs)

0.42178077369380856

# Use of symmetries for Trotter error calculation

We can leverage symmetries featured by molecular electronic Hamiltonians to carry out tighter estimations of Trotter error. The motivation is that in practice, unitary time propagation can be performed on an initial state belonging to one of the symmetry group irreducible representations. Thus, Trotter error can be estimated by considering only the subspace of the total Hilbert space corresponding to the irreducible representation of interest.


For concreteness, we define, for a Hamiltonian decomposition of the form 
$$
H=\sum_{n}H_{n}
$$
and a set of M observables $\{O_{m}\}$ that satisfy $[H_{n},O_{m}]=0$, $\forall n,m$, a symmetry-projected Trotter error:
$$
\alpha_{\mathbf{Q}}=\sum_{n,m}||[H_{n},H_{m}]||_{\mathbf{Q}}
$$
where $||[H_{n},H_{m}]||_{\mathbf{Q}}=||[H_{n},H_{m}]\mathcal{P}^{q_{1}}_{1}\mathcal{P}^{q_{2}}_{2}\cdots\mathcal{P}^{q_{M}}_{M}||$, $\mathcal{P}^{q_{i}}_{i}$ being the projector to the manifold spanned by eigenvectors of $O_{i}$ with eigenvalue $q_{i}$.

In Tequila, we have implemented tools for the calculation of $\alpha_{\mathbf{Q}}$.

# <font size="4"> a) Fermionic-based Hamiltonian decomposition.</font>

We can build fast-forwardable Hamiltonian fragments that preserve symmetries of the total Hamiltonian such as particle-number and spin. In fact there are methods of Hamiltonian decomposition, based on its second-quantized form, that yield fragments featuring those symmetries, such as the so-called Low-Rank factorization ([npj Quantum Information **7**, 83 (2021)](https://www.nature.com/articles/s41534-021-00416-z)), and its generalization ([PRX Quantum **2**, 040320 (2021)](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.2.040320)). 


Below, we show how to compute $\alpha_{\mathbf{Q}}$ for the Low-Rank factorization technique, using particle number and spin symmetries.

In [81]:
def BuildNumSpinOps(nqubs):
    #Build the number and spin fermionic operators...
    norb=nqubs//2
    # Sx 
    Sx = openfermion.hamiltonians.sx_operator(norb)
    # Sy
    Sy =  openfermion.hamiltonians.sy_operator(norb)
    # Sz
    Sz = openfermion.hamiltonians.sz_operator(norb)
    # N
    # S squared operator
    S_sq = Sx**2+Sy**2+Sz**2
    Nop = openfermion.hamiltonians.number_operator(nqubs)
    
    return Nop,S_sq,Sz

In [84]:
DictFerm={}

#We store operators and associated eigenvalues in a dictionary.
Nop,S_sq,Sz=BuildNumSpinOps(nqubs)
DictFerm['SymOps']=[Nop,S_sq,Sz]
DictFerm['QNumbs']=[2,0,0] #eigenvalues of the particle-number, S^{2} and S_{z} that define the manifold the Trotter
                            #error is projected on

EstTrotErr(LRFrags, nqubs, SymDict=DictSym)

0.21089038684690425

# <font size="4"> a) Qubit-based Hamiltonian decomposition.</font>

Similarly, for methods based on the decomposition of the Hamiltonian in its qubit encoding, it is possible to find qubit symmetries that commute with all Hamiltonian fragments afforded by a qubit-based decomposition. We can use a classically efficient technique based on qubit tapering (https://arxiv.org/abs/1701.08213) to find single Pauli product symmetries, which constitute a subset of all possible qubit symmetries that commute with Hamiltonian fragments.

In [79]:
#We import the function that finds qubit symmetries..
from tequila.trotter_err.qubtap_utils import GetQubSymsandEigs

In [80]:
SymOps,EigSyms=GetQubSymsandEigs(Hq,4)

#Symmetries are stored in a dictionary for calculation of Trotter error...
DictSym={}
DictSym['SymOps']=SymOps 
DictSym['QNumbs']=EigSyms #these are the eigenvalues of the qubit symmetry operators associated to the
                          #ground state of the Hamiltonian

#We use the FCSI Hamiltonian fragments found above, and compute Trotter error using symmetries
EstTrotErr(FCSIFrags, nqubs, SymDict=DictSym)

0.21058847648473317

An assessment and analysis of various Hamiltonian decomposition techniques in terms of their associated Trotter error using the tools presented in this tutorial can be found in (https://arxiv.org/abs/2210.10189). Script tests/testTrotErr.py computes Trotter errors for molecules and methods explored in that work.

# Trotter errors for different molecules and partition methods

Here, we show the calculation of upper bound estimations for Trotter error for H2 and LiH, using pre-computed Hamiltonian partitionings

In [49]:
import pickle
from tequila.trotter_err.qubtap_utils import GetQubSymsandEigs
import pandas as pd
from matplotlib import pyplot as plt



In [50]:
path_Frags='./Frag_Lib/'
path_Hams='./ham_lib/'


In [51]:
def loadFrags(mol,meth,path_Frags=path_Frags):
    '''
    Utility to load pre-calculated fragments.
    Input: mol, string for name of molecule. It can be either h2,lih,beh2,h2o,nh3
    meth: name of Hamiltonian partition method.
    path_Frags: path to location where Hamiltonian fragments are stored.
    '''
    #FileName=path_Frags+meth+'/'+mol+'_'+meth+'Frags'
    FileName=path_Frags+meth+'/'+mol+'_'+meth+'FiltFrags'
    f=open(FileName,'rb')
    dat=pickle.load(f)

    nqubs=dat['n_qubits']
    Frags=dat['grouping']

    f.close()

    return nqubs,Frags

def load_hamiltonian(mol_name,pathham=path_Hams):

    with open(pathham + mol_name + '_fer.bin', 'rb') as f:
        Hf = pickle.load(f)

    return Hf

def BuildNumSpinOps(nqubs):
    #Build the number and spin fermionic operators...
    norb=nqubs//2
    # Sx
    Sx = openfermion.hamiltonians.sx_operator(norb)
    # Sy
    Sy =  openfermion.hamiltonians.sy_operator(norb)
    # Sz
    Sz = openfermion.hamiltonians.sz_operator(norb)
    # N
    # S squared operator
    S_sq = Sx**2+Sy**2+Sz**2
    Nop = openfermion.hamiltonians.number_operator(nqubs)

    return Nop,S_sq,Sz



In [ ]:

mols=['h2','lih']
Dictnelecs={}
Dictnelecs['h2']=2
Dictnelecs['lih']=4

meths=['FC-SI','LR','LR-LCU','GFRO','GFROLCU','SD-GFRO','FRO','FC-LF']

GlobDict={}
GlobDict['mol']=[]
GlobDict['nqubs']=[]
GlobDict['method']=[]
GlobDict['Nfrags']=[]
GlobDict['alpha']=[]
GlobDict['sym_alpha']=[]


for mol in mols:
    for meth in meths:

        hferm=load_hamiltonian(mol)

        nqubs,Frags=loadFrags(mol,meth)
        print("Current molecule:",mol)
        print("Current method:",meth)
        #'Bare' trotter errors...
        alpha_2=EstTrotErr(Frags, nqubs)

        #symmetry-projected results...
        if meth=='FC-SI':
            bkHam=openfermion.bravyi_kitaev(hferm)
            tq_bkHam=QubitHamiltonian.from_openfermion(bkHam)

            SymOps,EigSyms=GetQubSymsandEigs(tq_bkHam,nqubs)

            DictSym={}
            DictSym['SymOps']=SymOps
            DictSym['QNumbs']=EigSyms

        else:
            DictSym={}
            nelec=Dictnelecs[mol]
            Nop,S_sq,Sz=BuildNumSpinOps(nqubs)
            DictSym['SymOps']=[Nop,S_sq,Sz]
            DictSym['QNumbs']=[nelec,0,0]

        alpha_2_sym=EstTrotErr(Frags, nqubs, SymDict=DictSym)

        GlobDict['mol'].append(mol)
        GlobDict['method'].append(meth)
        GlobDict['nqubs'].append(nqubs)
        GlobDict['Nfrags'].append(len(Frags))
        GlobDict['alpha'].append(alpha_2)
        GlobDict['sym_alpha'].append(alpha_2_sym)

pdResults=pd.DataFrame(GlobDict)


width = 0.8
Nmols=len(mols)
indices = np.arange(Nmols)

#xticks=[r'H$_2$','LiH',r'BeH$_{2}$',r'H$_{2}$O',r'NH$_{3}$']
xticks=mols

#group results in two sets...
Meths1=['FC-SI','LR','LR-LCU','GFRO','GFROLCU','SD-GFRO']
Meths2=['FRO','FC-LF']

fig, axs = plt.subplots(2, 2)
#Plot 1...
for meth in Meths2:
    meth_rows=pdResults.loc[pdResults['method'] == meth]

    axs[0,0].scatter(np.arange(Nmols),meth_rows['alpha'],label=meth)
    axs[0,0].plot(np.arange(Nmols),meth_rows['alpha'])


axs[0, 0].set_xticks(np.arange(Nmols))
axs[0, 0].set_xticklabels(xticks,fontsize=12)


axs[0,0].legend(fontsize=4,loc='upper left')
axs[0,0].set_title('Trotter errors',fontsize=12)
axs[0,0].set_ylabel(r'$\alpha$',fontsize=18)


#Plot 2...
for meth in Meths1:
    meth_rows=pdResults.loc[pdResults['method'] == meth]

    axs[1,0].scatter(np.arange(Nmols),meth_rows['alpha'],label=meth)
    axs[1,0].plot(np.arange(Nmols),meth_rows['alpha'])

axs[1, 0].set_xticks(np.arange(Nmols))
axs[1, 0].set_xticklabels(xticks,fontsize=12)


axs[1,0].legend(fontsize=4,loc='upper left')
axs[1,0].set_title('Trotter errors',fontsize=12)
axs[1,0].set_ylabel(r'$\alpha$',fontsize=18)

#Plot 3...
for meth in Meths2:
    meth_rows=pdResults.loc[pdResults['method'] == meth]

    axs[0,1].scatter(np.arange(Nmols),meth_rows['sym_alpha'],label=meth)
    axs[0,1].plot(np.arange(Nmols),meth_rows['sym_alpha'])


axs[0, 1].set_xticks(np.arange(Nmols))
axs[0, 1].set_xticklabels(xticks,fontsize=12)

axs[0,1].legend(fontsize=4,loc='upper left')
axs[0,1].set_title('Symmetry Projected Trotter errors',fontsize=10)
axs[0,1].set_ylabel(r'$\alpha$',fontsize=18)

#Plot 4...
for meth in Meths1:
    meth_rows=pdResults.loc[pdResults['method'] == meth]

    axs[1,1].scatter(np.arange(Nmols),meth_rows['sym_alpha'],label=meth)
    axs[1,1].plot(np.arange(Nmols),meth_rows['sym_alpha'])


axs[1, 1].set_xticks(np.arange(Nmols))
axs[1, 1].set_xticklabels(xticks,fontsize=12)


axs[1,1].legend(fontsize=4,loc='upper left')
axs[1,1].set_title('Symmetry Projected Trotter errors',fontsize=10)
axs[1,1].set_ylabel(r'$\alpha$',fontsize=18)



fig.tight_layout()
#plt.savefig("./AlphasPlots.pdf",bbox_inches='tight', dpi=150)
plt.show()